Заполните данные о необходимых переменных:

In [4]:
Today = '29/02/24'
Mounth = 'Feb'

Group = 'SUEK'
# Group = 'EUROCHEM'

In [126]:
from datetime import date
Output_file = "_".join([str(date.today()), Group, 'SAP', 'Debt', f'{Mounth}.xlsx'])
Sheet_in_output_file = 'Debt'
Output_file

'2024-03-21_SUEK_SAP_Debt_Feb.xlsx'

Открываю следующие файлы:

In [2]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column

In [3]:
query = """SELECT * FROM "RISKACCESS"."sapPositionArrears" WHERE "reportDate" = TO_DATE('29/02/24', 'DD/MM/YY') AND "productType" in (130,131,132,133)"""
data_export = export_from_RISKCUSTOM(query)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:78: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [13]:
import numpy as np

data_Debt = data_export.loc[data_export['productTypeName'].isin(['Транш по займам', 'Транш по кредитам'])].reset_index(drop=True)

data_Debt = data_Debt.groupby(['companyName', 'relationshipPartner',
       'businessPartnerName', 'creditConditionCurrency', 'termEnd']).agg({'purchaseValueDisplayCurrency' : sum,
                                                                          'purchaseValuePositionCurrency' : sum}).reset_index()

data_Debt['purchaseValuePositionCurrency'] = data_Debt['purchaseValuePositionCurrency'] * np.sign(data_Debt['purchaseValueDisplayCurrency'])
data_Debt = add_in_currency_column(data_Debt, col_with_CCY='creditConditionCurrency', col_with_VAL='purchaseValuePositionCurrency', day_for_export="29/02/24", CCY_to='USD')

data_Debt = data_Debt.rename(columns={"businessPartnerName": "Counterparty", "companyName": "Entity", "creditConditionCurrency": "Currency", "purchaseValuePositionCurrency": "amoutn_outstanding", "in_USD": "amount_USD_eq", 'purchaseValuePositionCurrency_in_USD':'amount_USD_eq'}).reset_index(drop=True)

data_Debt['Entity_code'] = merge_Mapping(data_Debt, col='Entity')
data_Debt['Entity_group'] = merge_SalesUnits(data_Debt, col='Entity_code', merge_col='ocpSegment')
data_Debt['Counterparty_code'] = merge_Mapping(data_Debt, col='Counterparty')
data_Debt['Counterparty_Group'] = merge_SalesUnits(data_Debt, col='Counterparty_code', merge_col='ocpSegment')

data_Debt = Period(data_Debt, day_for_count=Today, col_with_date='termEnd')

data_Debt['Holding'] = merge_SalesUnits(data_Debt, col='Entity_code', merge_col='holding')
data_Debt = data_Debt.query('Holding == @Group').reset_index(drop=True)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_15992\3454748448.py:6: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  'businessPartnerName', 'creditConditionCurrency', 'termEnd']).agg({'purchaseValueDisplayCurrency' : sum,
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_15992\3454748448.py:6: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  'businessPartnerName', 'creditConditionCurrency', 'termEnd']).agg({'purchaseValueDisplayCurrency' : sum,
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:78: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 

In [14]:

manual1 = data_Debt.loc[data_Debt.Entity_code == 'External' ,['Entity', 'Entity_code']].drop_duplicates()
manual2 = data_Debt.loc[data_Debt.Counterparty_code == 'External' ,['Counterparty', 'Counterparty_Group']].drop_duplicates()
manual1

,Entity,Entity_code


Записываю данные в output file:

In [15]:
new_columns = ['dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']
data_Debt = data_Debt.reindex(columns=(data_Debt.columns.tolist() + new_columns))
data_Debt['Source'] = 'SAP'

data = data_Debt[['Entity', 'Entity_group', 'Counterparty', 'Counterparty_Group', 'amoutn_outstanding', 'Currency', 'amount_USD_eq', 'termEnd', 'Days', 'Period', 'dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]

In [16]:
data.head(2)

,Entity,Entity_group,Counterparty,Counterparty_Group,amoutn_outstanding,Currency,amount_USD_eq,termEnd,Days,Period,dealClass1,dealClass2,instrumentOwner,dealSet,facility,Source
0,Black Sand Commodities,SUEK INT,AIM Capital Ltd,External,2.524600e+07,USD,2.524600e+07,2026-09-15,929.0,2Y+,NaN,NaN,NaN,NaN,NaN,SAP
1,Black Sand Commodities,SUEK INT,Erglis Limited,External,1.302614e+09,RUB,1.428303e+07,2029-10-08,2048.0,2Y+,NaN,NaN,NaN,NaN,NaN,SAP


In [131]:
data.to_excel(Output_file, sheet_name = Sheet_in_output_file, index = False)

Проверка на наличие тех же записей в Quantum:

In [110]:
quantum_data_export = pd.read_excel('C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP\\Quantum\\2024-03-21_EuroChem_quantum_Debt_Feb.xlsx')
quantum_data_export.head(2)

,entity,Entity_group,counterparty,Counterparty_Group,amoutn outstanding,Currency,amount USD eq,termEnd,Days,Period,dealClass1,dealClass2,instrumentOwner,dealSet,facility,Source
0,BRHRG,SAMSALES,Morgan Stanley BR,External,-8011.33,BRL,-1616.101732,2031-12-31,2862,2Y+,Loan,MM,EC_BORROWINGS,-,HRG Judicial recovery BRL,Quantum
1,BRHRG,SAMSALES,Morgan Stanley BR,External,-9958.01,BRL,-2008.799688,2032-12-31,3228,2Y+,Loan,MM,EC_BORROWINGS,-,HRG Judicial recovery BRL,Quantum


In [118]:
quantum_data_export['E_Cgroup'] = quantum_data_export.entity + quantum_data_export.Counterparty_Group + quantum_data_export.Currency + quantum_data_export.termEnd.astype(str) + quantum_data_export['amoutn outstanding'].abs().astype(str)
data_Debt['E_Cgroup'] = data_Debt.Entity_code + data_Debt.Counterparty_Group + data_Debt.Currency + data_Debt.termEnd.astype(str) + data_Debt.amoutn_outstanding.abs().astype(str)

iner_list = list(set(data_Debt['E_Cgroup']).intersection(set(quantum_data_export.E_Cgroup)))

SAP_iner = data_Debt[data_Debt.E_Cgroup.isin(iner_list)]
Quantum_iner = quantum_data_export[quantum_data_export.E_Cgroup.isin(iner_list)]

In [17]:
iner_list_2 = SAP_iner.index.tolist()

SAP_iner = data_Debt[data_Debt.index.isin(iner_list_2)]
data_Debt_not_iner = data_Debt[~data_Debt.index.isin(iner_list_2)]

SAP_iner.shape, Quantum_iner.shape

NameError: name 'SAP_iner' is not defined

In [123]:
SAP_iner = SAP_iner[['Entity', 'Entity_group', 'Counterparty', 'Counterparty_Group', 'amoutn_outstanding', 'Currency', 'amount_USD_eq', 'termEnd', 'Days', 'Period', 'dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]
data_Debt_not_iner = data_Debt_not_iner[['Entity', 'Entity_group', 'Counterparty', 'Counterparty_Group', 'amoutn_outstanding', 'Currency', 'amount_USD_eq', 'termEnd', 'Days', 'Period', 'dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]


In [124]:
data_Debt_not_iner.to_excel(Output_file, sheet_name=Sheet_in_output_file, index=False)
new_list(SAP_iner, Output_file, 'Inersection')
new_list(quantum_data_export, Output_file, 'Debt_Quantum')